In [ ]:
import pandas as pd
df_class_simple = pd.read_csv('breast_cancer.csv')
df_class_med = pd.read_csv('heart_disease.csv')
df_class_complex = pd.read_csv('diabetes.csv')

df_reg_simple = pd.read_csv('housing.csv')
df_reg_med = pd.read_csv('real_estate_valuation.csv')
df_reg_complex = pd.read_csv('Housing.csv')

In [ ]:
# Configure matplotlib for Jupyter notebooks
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
%matplotlib inline

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, f1_score, recall_score,confusion_matrix, classification_report, r2_score



class ClassificationNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ClassificationNeuralNetwork, self).__init__()
        self.task_type = 'classification'
        self.output_size = output_size

        # Hidden layer + activation
        self.hidden = nn.Linear(input_size, hidden_size)
        self.hidden_act = nn.LeakyReLU()

        # Output layer
        if output_size == 2:
            # Binary classification -> single sigmoid output
            self.output = nn.Linear(hidden_size, 1)
            self.final_act = nn.Sigmoid()

    def forward(self, x):
        # Hidden layer transformation
        x = self.hidden_act(self.hidden(x))
        # Output layer + final activation
        x = self.final_act(self.output(x))
        return x

    

class RegressionNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionNeuralNetwork, self).__init__()
        self.task_type = 'regression'
        
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, 1)  # Single output for regression
        self.lrelu = nn.LeakyReLU()

    def forward(self, x):
        x = self.hidden(x)      # Input to hidden layer
        x = self.lrelu(x)       # Apply Leaky ReLU activation
        x = self.output(x)      # Linear output (no activation)
        return x



def train_passive_sgd(model, X_train, y_train, X_val, y_val, 
                     epochs=100, learning_rate=0.01, weight_decay=0.001, verbose=True):
    # Make COPIES to avoid modifying original tensors
    y_train_work = y_train.clone()
    y_val_work = y_val.clone()
    
    # Choose loss function based on task type and output size
    if model.task_type == 'classification':
        if model.output.out_features == 1:
            criterion = nn.BCELoss()  # Binary classification
            # Convert targets to float for BCE (use working copies)
            y_train_work = y_train_work.float().unsqueeze(1)
            y_val_work = y_val_work.float().unsqueeze(1)
        else:
            criterion = nn.CrossEntropyLoss()  # Multi-class classification
    else:
        criterion = nn.MSELoss()  

    # SGD optimizer with weight decay (L2 regularization)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_train)
        train_loss = criterion(outputs, y_train_work)
        
        # Backward pass and optimization
        train_loss.backward()
        optimizer.step()
        
        # Validation phase
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val_work)
        
        train_losses.append(train_loss.item())
        val_losses.append(val_loss.item())
        
        # Print progress every 20 epochs (only if verbose=True)
        if verbose and (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
    
    return train_losses, val_losses

def prepare_data_for_torch(df, target_column, task_type='classification', test_size=0.2, val_size=0.1):
    # Separate features and target
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values
    
    # Handle different target types
    if task_type == 'classification':
        # Encode categorical targets to integers
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)
        num_classes = len(np.unique(y))
        print(f"Classification task: {num_classes} classes")
    else:
        print("Regression task")
        num_classes = 1

    # Split data into train, validation, and test sets  
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)
    
    # Scale features to [0, 1] range
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    X_train = torch.FloatTensor(X_train)
    X_val = torch.FloatTensor(X_val)
    X_test = torch.FloatTensor(X_test)
    
    if task_type == 'classification':
        y_train = torch.LongTensor(y_train)
        y_val = torch.LongTensor(y_val)
        y_test = torch.LongTensor(y_test)
    else:
        y_train = torch.FloatTensor(y_train).view(-1, 1)
        y_val = torch.FloatTensor(y_val).view(-1, 1)
        y_test = torch.FloatTensor(y_test).view(-1, 1)
    
    print(f"Data splits - Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")
    
    return (X_train, X_val, X_test, y_train, y_val, y_test), scaler, num_classes

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        
        if model.task_type == 'classification':
            # Binary classification with BCELoss
            predicted = (outputs > 0.5).float()
            y_test_np = y_test.cpu().numpy()
            predicted_np = predicted.cpu().numpy().flatten()
            
            accuracy = accuracy_score(y_test_np, predicted_np)
            precision = precision_score(y_test_np, predicted_np, average='weighted', zero_division=0)
            f1 = f1_score(y_test_np, predicted_np, average='weighted', zero_division=0)
            recall = recall_score(y_test_np, predicted_np, average='weighted', zero_division=0)
            confusion = confusion_matrix(y_test_np, predicted_np)
            class_report = classification_report(y_test_np, predicted_np)

            print(f"Test Accuracy: {accuracy:.4f}")
            print(f"Test Precision: {precision:.4f}")
            print(f"Test F1 Score: {f1:.4f}")
            print(f"Test Recall: {recall:.4f}")
            print(f"Confusion Matrix:\n{confusion}")
            print(f"Classification Report:\n{class_report}")
            return accuracy
        else:
            # Calculate MSE for regression
            mse = mean_squared_error(y_test.cpu().numpy(), outputs.cpu().numpy())
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test.cpu().numpy(), outputs.cpu().numpy())
            print(f"Test R^2: {r2:.4f}")
            # Calculate Pearson correlation coefficient
            correlation_matrix = np.corrcoef(y_test.cpu().numpy().flatten(), outputs.cpu().numpy().flatten())
            pearson_correlation = correlation_matrix[0, 1]
            print(f"Test RMSE: {rmse:.4f}")
            print(f"Test Pearson Correlation: {pearson_correlation:.4f}")
            print(f"Test MSE: {mse:.4f}")
            return mse

def visualise_losses(train_losses, val_losses, title="Loss over Epochs"):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)

In [ ]:
# Prepare simple classification data - Breast Cancer Wisconsin dataset
df_class_simple_processed = df_class_simple.copy()

# Check for missing values
missing_count = df_class_simple_processed.isnull().sum().sum()
if missing_count > 0:
    print(f"Handling {missing_count} missing values")
    # Fill missing values with median for numeric columns
    for col in df_class_simple_processed.columns:
        if df_class_simple_processed[col].isnull().any():
            if df_class_simple_processed[col].dtype in ['float64', 'int64']:
                df_class_simple_processed[col].fillna(df_class_simple_processed[col].median(), inplace=True)
else:
    print("No missing values found")

# The target column is 'Diagnosis' (M = Malignant, B = Benign)
target_column = 'Diagnosis'

# Convert target to binary (0 for Benign, 1 for Malignant)
df_class_simple_processed[target_column] = (df_class_simple_processed[target_column] == 'M').astype(int)

# All other columns are numeric features (no categorical encoding needed)
target_dist = df_class_simple_processed[target_column].value_counts().to_dict()
print(f"Breast cancer dataset prepared: {df_class_simple_processed.shape[0]} samples, {df_class_simple_processed.shape[1]-1} features")
print(f"Target distribution: {target_dist} (0=Benign, 1=Malignant)")

# Prepare data for PyTorch
target_col = 'Diagnosis'
splits, scaler, num_classes = prepare_data_for_torch(
    df_class_simple_processed, 
    target_col, 
    task_type='classification'
)

X_train, X_val, X_test, y_train, y_val, y_test = splits

input_size = X_train.shape[1]
hidden_size = 32
output_size = num_classes
print(f"Input size: {input_size}, Hidden size: {hidden_size}, Output size: {output_size}")

# Create and train classification model
baseline_model = ClassificationNeuralNetwork(input_size, hidden_size, output_size)
print(f"Model created with {sum(p.numel() for p in baseline_model.parameters())} parameters")

print("\n--- Training Baseline Model ---")
train_losses, val_losses = train_passive_sgd(
    baseline_model, X_train, y_train, X_val, y_val,
    epochs=1500,
    learning_rate=0.01,
    weight_decay=0.0001
)


print("\n--- Final Evaluation ---")
test_accuracy = evaluate_model(baseline_model, X_test, y_test)
visualise_losses(train_losses, val_losses, title="Breast Cancer Classification Loss over Epochs")

In [ ]:
from sklearn.preprocessing import StandardScaler

# Handle missing values more thoroughly
df_reg_simple = df_reg_simple.copy()
df_reg_simple['total_bedrooms'].fillna(df_reg_simple['total_bedrooms'].median(), inplace=True)

# Fill any other missing values
for col in df_reg_simple.columns:
    if df_reg_simple[col].isnull().any():
        if df_reg_simple[col].dtype in ['float64', 'int64']:
            df_reg_simple[col].fillna(df_reg_simple[col].median(), inplace=True)
        else:
            df_reg_simple[col].fillna(df_reg_simple[col].mode()[0], inplace=True)

reg_target = 'median_house_value'
# Prepare regression dataset
df_reg_processed = df_reg_simple.copy()

# Handle any categorical columns in housing data
for col in df_reg_processed.columns:
    if df_reg_processed[col].dtype == 'object' and col != reg_target:
        print(f"Encoding categorical column: {col}")
        df_reg_processed[col] = LabelEncoder().fit_transform(df_reg_processed[col].astype(str))

# Scale the target values to prevent large gradients
target_scaler = StandardScaler()
df_reg_processed[reg_target] = target_scaler.fit_transform(df_reg_processed[[reg_target]])
print(f"Dataset prepared: {df_reg_processed.shape[0]} samples, target scaled")

reg_splits, reg_scaler, _ = prepare_data_for_torch(
    df_reg_processed, 
    reg_target, 
    task_type='regression'
)

X_train_reg, X_val_reg, X_test_reg, y_train_reg, y_val_reg, y_test_reg = reg_splits

print(f"\n--- Regression Model Configuration ---")
input_size_reg = X_train_reg.shape[1]
hidden_size_reg = 64  # Overestimate for regularization
output_size_reg = 1   # Single output for regression
print(f"Input features: {input_size_reg}")

# Create regression model
reg_model = RegressionNeuralNetwork(input_size_reg, hidden_size_reg)
print(f"Regression model created with {sum(p.numel() for p in reg_model.parameters())} parameters")

print("\n--- Training Regression Baseline Model ---")

# Train the regression model
reg_train_losses, reg_val_losses = train_passive_sgd(
    reg_model, X_train_reg, y_train_reg, X_val_reg, y_val_reg,
    epochs=1500,
    learning_rate=0.01,
    weight_decay=0.01
)


print("\n--- Regression Final Evaluation ---")
test_mse = evaluate_model(reg_model, X_test_reg, y_test_reg)

visualise_losses(reg_train_losses, reg_val_losses, title="Regression Loss over Epochs")


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
# Prepare classification data - Heart Disease dataset
df_class_med_processed = df_class_med.copy()

# Check for missing values and handle them
missing_count = df_class_med_processed.isnull().sum().sum()
if missing_count > 0:
    print(f"Handling {missing_count} missing values")
    # Fill missing values appropriately 
    for col in df_class_med_processed.columns:
        if df_class_med_processed[col].isnull().any():
            if df_class_med_processed[col].dtype in ['float64', 'int64']:
                df_class_med_processed[col].fillna(df_class_med_processed[col].median(), inplace=True)
            else:
                df_class_med_processed[col].fillna(df_class_med_processed[col].mode()[0], inplace=True)
else:
    print("No missing values found")

# The target variable is 'num' (0 = no heart disease, >0 = heart disease)
target_column = 'num'

# Convert multi-class target to binary classification (0 vs >0)
df_class_med_processed[target_column] = (df_class_med_processed[target_column] > 0).astype(int)

# All features are already numeric (no categorical encoding needed)
target_dist = df_class_med_processed[target_column].value_counts().to_dict()
print(f"Heart disease dataset prepared: {df_class_med_processed.shape[0]} samples, {df_class_med_processed.shape[1]-1} features")
print(f"Target distribution: {target_dist} (0=No Disease, 1=Disease)")

# Prepare data for PyTorch
target_col = 'num'
splits, scaler, num_classes = prepare_data_for_torch(
    df_class_med_processed, 
    target_col, 
    task_type='classification'
)

X_train, X_val, X_test, y_train, y_val, y_test = splits

print(f"\n--- Model Configuration ---")
input_size = X_train.shape[1]
hidden_size = 256
output_size = num_classes
print(f"Input size: {input_size}, Hidden size: {hidden_size}, Output size: {output_size}")

# Create and train classification model
baseline_model = ClassificationNeuralNetwork(input_size, hidden_size, output_size)
print(f"Model created with {sum(p.numel() for p in baseline_model.parameters())} parameters")

print("\n--- Training Baseline Model ---")
train_losses, val_losses = train_passive_sgd(
    baseline_model, X_train, y_train, X_val, y_val,
    epochs=1000,
    learning_rate=0.01,
    weight_decay=0.01
)


print("\n--- Final Evaluation ---")
test_accuracy = evaluate_model(baseline_model, X_test, y_test)
visualise_losses(train_losses, val_losses, title="Heart Disease Classification Loss over Epochs")


In [ ]:
from sklearn.preprocessing import StandardScaler
# Prepare regression data - real estate valuation dataset
df_reg_med_processed = df_reg_med.copy()

# Check for missing values (should be none in this dataset)
if df_reg_med_processed.isnull().sum().sum() > 0:
    print("Missing values found - handling them")
    print(df_reg_med_processed.isnull().sum())
else:
    print("No missing values found")

# No missing values in this dataset, but let's handle any that might exist
for col in df_reg_med_processed.columns:
    if df_reg_med_processed[col].isnull().any():
        if df_reg_med_processed[col].dtype in ['float64', 'int64']:
            df_reg_med_processed[col].fillna(df_reg_med_processed[col].median(), inplace=True)
        else:
            df_reg_med_processed[col].fillna(df_reg_med_processed[col].mode()[0], inplace=True)

# The target column for this real estate dataset
reg_target = 'Y house price of unit area'
# Prepare regression dataset - all features are already numeric
df_reg_med_processed = df_reg_med_processed.copy()

# Check for any categorical columns 
categorical_columns = df_reg_med_processed.select_dtypes(include=['object']).columns.tolist()
if categorical_columns:
    print(f"Found categorical columns: {categorical_columns}")
    for col in categorical_columns:
        if col != reg_target:
            df_reg_med_processed[col] = LabelEncoder().fit_transform(df_reg_med_processed[col].astype(str))

# Scale the target values to prevent large gradients and improve training stability
target_scaler = StandardScaler()
df_reg_med_processed[reg_target] = target_scaler.fit_transform(df_reg_med_processed[[reg_target]])
print(f"Dataset prepared: {df_reg_med_processed.shape[0]} samples, {df_reg_med_processed.shape[1]-1} features")

reg_splits, reg_scaler, _ = prepare_data_for_torch(
    df_reg_med_processed, 
    reg_target, 
    task_type='regression'
)

X_train_reg, X_val_reg, X_test_reg, y_train_reg, y_val_reg, y_test_reg = reg_splits

print(f"\n--- Regression Model Configuration ---")
input_size_reg = X_train_reg.shape[1]
hidden_size_reg = 64  # Overestimate for regularization
output_size_reg = 1   # Single output for regression
print(f"Input features: {input_size_reg}")
# Create regression model
reg_model = RegressionNeuralNetwork(input_size_reg, hidden_size_reg)
print(f"Regression model created with {sum(p.numel() for p in reg_model.parameters())} parameters")

print("\n--- Training Regression Baseline Model ---")

# Train the regression model
reg_train_losses, reg_val_losses = train_passive_sgd(
    reg_model, X_train_reg, y_train_reg, X_val_reg, y_val_reg,
    epochs=1500,
    learning_rate=0.01,
    weight_decay=0.01
)



print("\n--- Regression Final Evaluation ---")
test_mse = evaluate_model(reg_model, X_test_reg, y_test_reg)

visualise_losses(reg_train_losses, reg_val_losses, title="Regression Loss over Epochs")


In [ ]:
# Prepare complex classification data - Diabetes dataset
df_class_complex_processed = df_class_complex.copy()

# Columns where 0 is not medically possible and indicates missing data
zero_not_acceptable = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Replace 0s with NaN for these columns, then impute with median
for col in zero_not_acceptable:
    if col in df_class_complex_processed.columns:
        # Replace 0 with NaN (except for Insulin where 0 might be acceptable)
        if col != 'Insulin':
            df_class_complex_processed[col] = df_class_complex_processed[col].replace(0, np.nan)
        else:
            # For Insulin, only replace very low values that seem unrealistic
            df_class_complex_processed[col] = df_class_complex_processed[col].replace(0, np.nan)
        
        # Impute missing values with median
        df_class_complex_processed[col].fillna(df_class_complex_processed[col].median(), inplace=True)

# Check final missing values
missing_count = df_class_complex_processed.isnull().sum().sum()
if missing_count > 0:
    print(f"Remaining {missing_count} missing values after preprocessing")
    df_class_complex_processed = df_class_complex_processed.dropna()
else:
    print("All missing values handled successfully")

# Target variable is 'Outcome' (0 = no diabetes, 1 = diabetes)
target_column = 'Outcome'
target_dist = df_class_complex_processed[target_column].value_counts().to_dict()
print(f"Diabetes dataset prepared: {df_class_complex_processed.shape[0]} samples, {df_class_complex_processed.shape[1]-1} features")
print(f"Target distribution: {target_dist} (0=No Diabetes, 1=Diabetes)")
print(f"Dataset imbalance: {target_dist[0]/(target_dist[0]+target_dist[1]):.1%} no diabetes, {target_dist[1]/(target_dist[0]+target_dist[1]):.1%} diabetes")

# Prepare data for PyTorch
target_col = 'Outcome'
splits, scaler, num_classes = prepare_data_for_torch(
    df_class_complex_processed, 
    target_col, 
    task_type='classification'
)

X_train, X_val, X_test, y_train, y_val, y_test = splits

print(f"\n--- Training Complex Classification Model ---")
input_size = X_train.shape[1]
hidden_size = 128  # Larger hidden layer for complex dataset
output_size = num_classes
print(f"Input size: {input_size}, Hidden size: {hidden_size}, Output size: {output_size}")

# Create and train classification model with larger capacity for complex problem
complex_model = ClassificationNeuralNetwork(input_size, hidden_size, output_size)
print(f"Complex model: {sum(p.numel() for p in complex_model.parameters())} parameters")

# Train with careful hyperparameters for medical data
train_losses, val_losses = train_passive_sgd(
    complex_model, X_train, y_train, X_val, y_val,
    epochs=1000,
    learning_rate=0.005,  # Lower learning rate for stability
    weight_decay=0.0001   # Moderate regularization
)

print("\n--- Final Evaluation ---")
test_accuracy = evaluate_model(complex_model, X_test, y_test)
visualise_losses(train_losses, val_losses, title="Diabetes Classification Loss over Epochs")

In [ ]:
df_reg_complex.head()

In [ ]:
# Prepare complex regression data - comprehensive housing features
df_reg_complex_processed = df_reg_complex.copy()

# Check for missing values
missing_count = df_reg_complex_processed.isnull().sum().sum()
if missing_count > 0:
    print(f"Handling {missing_count} missing values")
    # Handle missing values appropriately
    for col in df_reg_complex_processed.columns:
        if df_reg_complex_processed[col].isnull().any():
            if df_reg_complex_processed[col].dtype in ['float64', 'int64']:
                df_reg_complex_processed[col].fillna(df_reg_complex_processed[col].median(), inplace=True)
            else:
                df_reg_complex_processed[col].fillna(df_reg_complex_processed[col].mode()[0], inplace=True)
else:
    print("No missing values found")

# Handle categorical variables - convert yes/no to binary and encode furnishing status
binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
for col in binary_cols:
    if col in df_reg_complex_processed.columns:
        df_reg_complex_processed[col] = (df_reg_complex_processed[col] == 'yes').astype(int)

# Handle furnishingstatus with label encoding (ordinal: unfurnished < semi-furnished < furnished)
if 'furnishingstatus' in df_reg_complex_processed.columns:
    furnishing_map = {'unfurnished': 0, 'semi-furnished': 1, 'furnished': 2}
    df_reg_complex_processed['furnishingstatus'] = df_reg_complex_processed['furnishingstatus'].map(furnishing_map)

# Target variable is price
target_column = 'price'
target_stats = df_reg_complex_processed[target_column].describe()
print(f"Complex dataset prepared: {df_reg_complex_processed.shape[0]} samples, {df_reg_complex_processed.shape[1]-1} features")
print(f"Target range: {target_stats['min']:.0f} to {target_stats['max']:.0f} (mean: {target_stats['mean']:.0f})")

# Scale the target values to prevent large gradients and improve training stability
target_scaler = StandardScaler()
df_reg_complex_processed[target_column] = target_scaler.fit_transform(df_reg_complex_processed[[target_column]])

# Prepare data for PyTorch
target_col = 'price'
splits, scaler, _ = prepare_data_for_torch(
    df_reg_complex_processed, 
    target_col, 
    task_type='regression'
)

X_train_reg, X_val_reg, X_test_reg, y_train_reg, y_val_reg, y_test_reg = splits

print(f"\n--- Training Complex Regression Model ---")
input_size_reg = X_train_reg.shape[1]
hidden_size_reg = 64  # Larger hidden layer for complex dataset
output_size_reg = 1   # Single output for regression
print(f"Input features: {input_size_reg}")


# Create regression model with larger capacity for complex problem
complex_reg_model = RegressionNeuralNetwork(input_size_reg, hidden_size_reg)
print(f"Complex regression model: {sum(p.numel() for p in complex_reg_model.parameters())} parameters, training for 1500 epochs")

# Train with more epochs and careful learning rate for complex problem
reg_train_losses, reg_val_losses = train_passive_sgd(
    complex_reg_model, X_train_reg, y_train_reg, X_val_reg, y_val_reg,
    epochs=1500,
    learning_rate=0.05,  # Careful learning rate for stability
    weight_decay=0.001   # Moderate regularization
)


print("\n--- Final Evaluation ---")
test_mse = evaluate_model(complex_reg_model, X_test_reg, y_test_reg)
visualise_losses(reg_train_losses, reg_val_losses, title="Complex Regression Loss over Epochs")

# Extensive Grid Search for All Datasets

This section performs comprehensive hyperparameter tuning for all classification and regression datasets using grid search to find optimal configurations.

In [ ]:
import json
import os
import time
from datetime import datetime
from itertools import product
import pandas as pd
from sklearn.metrics import f1_score, r2_score

def create_grid_search_params():
    # Classification grid - IMPROVED with wider ranges and longer training
    classification_grid = {
        'hidden_size': [32, 64, 128, 256],  # More sizes
        'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],  # Wider range
        'weight_decay': [0.0001, 0.001, 0.01],  # More options
        'epochs': [200, 500, 1000]  # Longer training to show differences
    }
    
    # Regression grid - similar improvements
    regression_grid = {
        'hidden_size': [32, 64, 128, 256],
        'learning_rate': [0.0001, 0.001, 0.01, 0.05],
        'weight_decay': [0.0, 0.0001, 0.001, 0.01], 
        'epochs': [500, 1000, 1500]  # Longer for regression
    }
    
    return classification_grid, regression_grid

def create_reduced_grid_search_params():
    
    # Reduced classification grid - for testing
    classification_grid = {
        'hidden_size': [64, 256],  
        'learning_rate': [0.001, 0.05],  # Extreme values to show difference
        'weight_decay': [0.0, 0.01],  # Extreme values
        'epochs': [200, 500]  # Reasonable epochs
    }
    
    # Reduced regression grid 
    regression_grid = {
        'hidden_size': [64, 256],
        'learning_rate': [0.001, 0.05],
        'weight_decay': [0.0, 0.01], 
        'epochs': [500, 1000]
    }
    
    return classification_grid, regression_grid


def save_results(results, filename, dataset_info):
    output = {
        'dataset_info': dataset_info,
        'timestamp': datetime.now().isoformat(),
        'total_experiments': len(results),
        'results': results
    }
    
    os.makedirs('results/passive', exist_ok=True)
    filepath = f'results/passive/{filename}'
    
    with open(filepath, 'w') as f:
        json.dump(output, f, indent=2)
    
    print(f"✅ Results saved to {filepath}")

def print_best_results(results, task_type, top_k=3):
    if task_type == 'classification':
        sorted_results = sorted(results, key=lambda x: x.get('test_f1', -float('inf')), reverse=True)
        metric = 'test_f1'
    else:
        sorted_results = sorted(results, key=lambda x: x.get('test_r2', -float('inf')), reverse=True)
        metric = 'test_r2'
    
    print(f"\n🏆 Top {top_k} Results (by {metric}):")
    print("=" * 80)
    
    for i, result in enumerate(sorted_results[:top_k]):
        print(f"\nRank {i+1}:")
        print(f"  Hidden Size: {result['hidden_size']}")
        print(f"  Learning Rate: {result['learning_rate']}")
        print(f"  Weight Decay: {result['weight_decay']}")
        print(f"  Epochs: {result['epochs']}")
        val_metric = result.get(metric, None)
        if val_metric is not None:
            print(f"  {metric.replace('_', ' ').title()}: {val_metric:.4f}")
        print(f"  Final Val Loss: {result['final_val_loss']:.4f}")
        print(f"  Training Time: {result.get('training_time', 0):.1f}s")

print("✅ Improved grid search utilities loaded!")

In [ ]:
def run_grid_search(X_train, X_val, X_test, y_train, y_val, y_test, 
                   task_type, num_classes=None, dataset_name="dataset"):
    classification_grid, regression_grid = create_grid_search_params()
    
    if task_type == 'classification':
        param_grid = classification_grid
    else:
        param_grid = regression_grid
    
    # Generate all parameter combinations
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    param_combinations = list(product(*param_values))
    
    print(f"\n🔍 Starting Grid Search for {dataset_name}")
    print(f"Task: {task_type.title()}")
    print(f"Total combinations to test: {len(param_combinations)}")
    print(f"Parameters: {param_names}")
    
    results = []
    start_time = time.time()
    
    # Store ORIGINAL data copies to avoid in-place modifications
    X_train_orig = X_train.clone()
    X_val_orig = X_val.clone() 
    X_test_orig = X_test.clone()
    y_train_orig = y_train.clone()
    y_val_orig = y_val.clone()
    y_test_orig = y_test.clone()
    
    for i, combination in enumerate(param_combinations):
        params = dict(zip(param_names, combination))
        
        print(f"\n[{i+1}/{len(param_combinations)}] Testing: {params}")
        
        try:
            # SET RANDOM SEED for reproducible results
            torch.manual_seed(42 + i)  # Different seed for each experiment
            np.random.seed(42 + i)
            
            # Create fresh copies of data for each experiment
            X_train_exp = X_train_orig.clone()
            X_val_exp = X_val_orig.clone()
            X_test_exp = X_test_orig.clone()
            y_train_exp = y_train_orig.clone()
            y_val_exp = y_val_orig.clone()
            y_test_exp = y_test_orig.clone()
            
            # Create NEW model for each experiment (fresh initialization)
            input_size = X_train_exp.shape[1]
            
            if task_type == 'classification':
                model = ClassificationNeuralNetwork(input_size, params['hidden_size'], num_classes)
            else:
                model = RegressionNeuralNetwork(input_size, params['hidden_size'])
            
            # Train model with current parameters using COPIES of the data (suppress epoch prints)
            train_losses, val_losses = train_passive_sgd(
                model, X_train_exp, y_train_exp, X_val_exp, y_val_exp,
                epochs=params['epochs'],
                learning_rate=params['learning_rate'],
                weight_decay=params['weight_decay'],
                verbose=False  # Suppress epoch prints during grid search
            )
            
            # Evaluate model using ORIGINAL unmodified targets
            model.eval()
            with torch.no_grad():
                # Validation and test predictions
                val_outputs = model(X_val_orig)
                test_outputs = model(X_test_orig)
                
                if task_type == 'classification':
                    if model.output.out_features == 1:
                        # Binary classification
                        val_pred = (val_outputs > 0.5).float().cpu().numpy().flatten()
                        test_pred = (test_outputs > 0.5).float().cpu().numpy().flatten()
                    else:
                        # Multi-class classification
                        _, val_pred = torch.max(val_outputs, 1)
                        _, test_pred = torch.max(test_outputs, 1)
                        val_pred = val_pred.cpu().numpy()
                        test_pred = test_pred.cpu().numpy()
                    
                    # Use ORIGINAL targets for evaluation
                    y_val_np = y_val_orig.cpu().numpy()
                    y_test_np = y_test_orig.cpu().numpy()
                    
                    # Calculate metrics
                    val_f1 = f1_score(y_val_np, val_pred, average='weighted', zero_division=0)
                    test_f1 = f1_score(y_test_np, test_pred, average='weighted', zero_division=0)
                    val_accuracy = accuracy_score(y_val_np, val_pred)
                    test_accuracy = accuracy_score(y_test_np, test_pred)
                    
                else:
                    # Regression
                    y_val_np = y_val_orig.cpu().numpy()
                    y_test_np = y_test_orig.cpu().numpy()
                    val_pred = val_outputs.cpu().numpy()
                    test_pred = test_outputs.cpu().numpy()
                    
                    # Calculate R² score
                    val_r2 = r2_score(y_val_np, val_pred)
                    test_r2 = r2_score(y_test_np, test_pred)
                    val_mse = mean_squared_error(y_val_np, val_pred)
                    test_mse = mean_squared_error(y_test_np, test_pred)
            
            # Store results
            result = {
                'hidden_size': params['hidden_size'],
                'learning_rate': params['learning_rate'],
                'weight_decay': params['weight_decay'],
                'epochs': params['epochs'],
                'final_train_loss': train_losses[-1],
                'final_val_loss': val_losses[-1],
                'training_time': time.time() - start_time
            }
            
            if task_type == 'classification':
                result.update({
                    'val_f1': val_f1,
                    'test_f1': test_f1,
                    'val_accuracy': val_accuracy,
                    'test_accuracy': test_accuracy
                })
                print(f"  Val F1: {val_f1:.4f}, Test F1: {test_f1:.4f}")
            else:
                result.update({
                    'val_r2': val_r2,
                    'test_r2': test_r2,
                    'val_mse': val_mse,
                    'test_mse': test_mse
                })
                print(f"  Val R²: {val_r2:.4f}, Test R²: {test_r2:.4f}")
                
            results.append(result)
            
        except Exception as e:
            print(f"  Error with params {params}: {str(e)}")
            continue
    
    total_time = time.time() - start_time
    print(f"\nGrid search completed in {total_time:.1f} seconds")
    print(f"Successfully tested {len(results)}/{len(param_combinations)} combinations")
    
    return results


In [ ]:

# Simple classification splits (breast_cancer.csv) - rename existing variables for consistency
print("\n--- Simple Classification Data Preparation ---")
sim_splits, sim_scaler, sim_num_classes = prepare_data_for_torch(
    df_class_simple_processed, 
    'Diagnosis',  # Updated target column for breast cancer
    task_type='classification'
)
X_train, X_val, X_test, y_train, y_val, y_test = sim_splits

# Medium classification splits (heart_disease.csv)
print("\n--- Medium Classification Data Preparation ---")
med_splits, med_scaler, med_num_classes = prepare_data_for_torch(
    df_class_med_processed, 
    'num',  # Updated target column for heart disease
    task_type='classification'
)
X_train_med, X_val_med, X_test_med, y_train_med, y_val_med, y_test_med = med_splits

# Complex classification splits (diabetes.csv) 
print("\n--- Complex Classification Data Preparation ---")
complex_splits, complex_scaler, complex_num_classes = prepare_data_for_torch(
    df_class_complex_processed,
    'Outcome',  # Updated target column for diabetes
    task_type='classification'
)
X_train_complex, X_val_complex, X_test_complex, y_train_complex, y_val_complex, y_test_complex = complex_splits

# CORRECT - Create fresh splits from the PROCESSED simple regression dataset
print("\n--- Simple Regression Data Preparation ---")
simple_reg_splits, simple_reg_scaler, _ = prepare_data_for_torch(
    df_reg_processed,  # Use the PROCESSED df_reg_processed (already handled categorical columns)
    'median_house_value',  # California housing target
    task_type='regression'
)
X_train_simple_reg, X_val_simple_reg, X_test_simple_reg, y_train_simple_reg, y_val_simple_reg, y_test_simple_reg = simple_reg_splits

# Medium regression splits (real_estate_valuation.csv)
print("\n--- Medium Regression Data Preparation ---") 
med_reg_splits, med_reg_scaler, _ = prepare_data_for_torch(
    df_reg_med_processed,
    'Y house price of unit area',
    task_type='regression'
)
X_train_med_reg, X_val_med_reg, X_test_med_reg, y_train_med_reg, y_val_med_reg, y_test_med_reg = med_reg_splits

# Complex regression splits (Housing.csv)
print("\n--- Complex Regression Data Preparation ---")
complex_reg_splits, complex_reg_scaler, _ = prepare_data_for_torch(
    df_reg_complex_processed,
    'price', 
    task_type='regression'
)
X_train_complex_reg, X_val_complex_reg, X_test_complex_reg, y_train_complex_reg, y_val_complex_reg, y_test_complex_reg = complex_reg_splits


In [ ]:



# Use the simple classification data splits (breast cancer)
simple_class_results = run_grid_search(
    X_train, X_val, X_test, y_train, y_val, y_test,
    task_type='classification',
    num_classes=sim_num_classes,
    dataset_name="Simple Classification (Breast Cancer)"
)

dataset_info = {
    'name': 'Simple Classification (Breast Cancer)', 
    'samples': X_train.shape[0] + X_val.shape[0] + X_test.shape[0],
    'features': X_train.shape[1],
    'classes': sim_num_classes
}

save_results(simple_class_results, 'simple_classification_grid_search.json', dataset_info)
print_best_results(simple_class_results, task_type='classification', top_k=5)

In [ ]:


# Use the medium complexity data splits
medium_class_results = run_grid_search(
    X_train_med, X_val_med, X_test_med, y_train_med, y_val_med, y_test_med,
    task_type='classification',
    num_classes=med_num_classes,
    dataset_name="Medium Classification (Heart Disease)"
)

dataset_info = {
    'name': 'Medium Classification (Heart Disease)', 
    'samples': X_train_med.shape[0] + X_val_med.shape[0] + X_test_med.shape[0],
    'features': X_train_med.shape[1],
    'classes': med_num_classes
}

save_results(medium_class_results, 'medium_classification_grid_search.json', dataset_info)

# === COMPLEX CLASSIFICATION (Diabetes) ===
print("\n" + "="*60)
print("COMPLEX CLASSIFICATION - Diabetes Dataset")
print("="*60)

complex_class_results = run_grid_search(
    X_train_complex, X_val_complex, X_test_complex, y_train_complex, y_val_complex, y_test_complex,
    task_type='classification',
    num_classes=complex_num_classes,
    dataset_name="Complex Classification (Diabetes)"
)

dataset_info = {
    'name': 'Complex Classification (Diabetes)',
    'samples': X_train_complex.shape[0] + X_val_complex.shape[0] + X_test_complex.shape[0],
    'features': X_train_complex.shape[1],
    'classes': complex_num_classes
}

save_results(complex_class_results, 'complex_classification_grid_search.json', dataset_info)

print_best_results(medium_class_results, 'classification', top_k=5)
print_best_results(complex_class_results, 'classification', top_k=5)

In [ ]:


simple_reg_results = run_grid_search(
    X_train_simple_reg, X_val_simple_reg, X_test_simple_reg, 
    y_train_simple_reg, y_val_simple_reg, y_test_simple_reg,
    task_type='regression',
    dataset_name="Simple Regression (California Housing)"
)

dataset_info = {
    'name': 'Simple Regression (California Housing)',
    'samples': X_train_simple_reg.shape[0] + X_val_simple_reg.shape[0] + X_test_simple_reg.shape[0],
    'features': X_train_simple_reg.shape[1]
}

save_results(simple_reg_results, 'simple_regression_grid_search.json', dataset_info)

# === MEDIUM REGRESSION (Real Estate Valuation) ===
print("\n" + "="*60)
print("MEDIUM REGRESSION - Real Estate Valuation Dataset")
print("="*60)

medium_reg_results = run_grid_search(
    X_train_med_reg, X_val_med_reg, X_test_med_reg,
    y_train_med_reg, y_val_med_reg, y_test_med_reg, 
    task_type='regression',
    dataset_name="Medium Regression (Real Estate Valuation)"
)

dataset_info = {
    'name': 'Medium Regression (Real Estate Valuation)',
    'samples': X_train_med_reg.shape[0] + X_val_med_reg.shape[0] + X_test_med_reg.shape[0],
    'features': X_train_med_reg.shape[1]
}

save_results(medium_reg_results, 'medium_regression_grid_search.json', dataset_info)

# === COMPLEX REGRESSION (Comprehensive Housing) ===
print("\n" + "="*60)
print("COMPLEX REGRESSION - Comprehensive Housing Dataset")
print("="*60)

complex_reg_results = run_grid_search(
    X_train_complex_reg, X_val_complex_reg, X_test_complex_reg,
    y_train_complex_reg, y_val_complex_reg, y_test_complex_reg,
    task_type='regression', 
    dataset_name="Complex Regression (Comprehensive Housing)"
)

dataset_info = {
    'name': 'Complex Regression (Comprehensive Housing)',
    'samples': X_train_complex_reg.shape[0] + X_val_complex_reg.shape[0] + X_test_complex_reg.shape[0],
    'features': X_train_complex_reg.shape[1]
}

save_results(complex_reg_results, 'complex_regression_grid_search.json', dataset_info)

print_best_results(simple_reg_results, 'regression', top_k=5)
print_best_results(medium_reg_results, 'regression', top_k=5)
print_best_results(complex_reg_results, 'regression', top_k=5)

# Multiple Runs for Statistical Analysis

This section runs each model configuration 10 times with different random seeds to collect statistical data for hypothesis testing.

In [ ]:
import json
import os
import time
from datetime import datetime

# Create results directory
os.makedirs('results/passive_multiple_runs', exist_ok=True)

def run_multiple_experiments(model_configs, n_runs=10):
    all_results = {}
    
    for config_name, config in model_configs.items():
        print(f"\n{'='*60}")
        print(f"Running {config_name} - {n_runs} times")
        print(f"{'='*60}")
        
        results = []
        
        for run in range(n_runs):
            print(f"\nRun {run + 1}/{n_runs} for {config_name}")
            
            # Set different random seed for each run
            torch.manual_seed(42 + run)
            np.random.seed(42 + run)
            
            try:
                # Get data splits
                X_train, X_val, X_test = config['data']['X_train'], config['data']['X_val'], config['data']['X_test']
                y_train, y_val, y_test = config['data']['y_train'], config['data']['y_val'], config['data']['y_test']
                
                # Create fresh model
                if config['task_type'] == 'classification':
                    model = ClassificationNeuralNetwork(
                        config['input_size'], 
                        config['hidden_size'], 
                        config['num_classes']
                    )
                else:
                    model = RegressionNeuralNetwork(
                        config['input_size'], 
                        config['hidden_size']
                    )
                
                # Train model
                start_time = time.time()
                train_losses, val_losses = train_passive_sgd(
                    model, X_train, y_train, X_val, y_val,
                    epochs=config['epochs'],
                    learning_rate=config['learning_rate'],
                    weight_decay=config['weight_decay'],
                    verbose=False
                )
                training_time = time.time() - start_time
                
                # Evaluate model
                model.eval()
                with torch.no_grad():
                    test_outputs = model(X_test)
                    
                    if config['task_type'] == 'classification':
                        if model.output.out_features == 1:
                            predicted = (test_outputs > 0.5).float().cpu().numpy().flatten()
                        else:
                            _, predicted = torch.max(test_outputs, 1)
                            predicted = predicted.cpu().numpy()
                        
                        y_test_np = y_test.cpu().numpy()
                        
                        accuracy = accuracy_score(y_test_np, predicted)
                        precision = precision_score(y_test_np, predicted, average='weighted', zero_division=0)
                        f1 = f1_score(y_test_np, predicted, average='weighted', zero_division=0)
                        recall = recall_score(y_test_np, predicted, average='weighted', zero_division=0)
                        
                        metrics = {
                            'accuracy': accuracy,
                            'precision': precision,
                            'f1_score': f1,
                            'recall': recall
                        }
                        
                    else:  # regression
                        y_test_np = y_test.cpu().numpy()
                        predictions = test_outputs.cpu().numpy()
                        
                        mse = mean_squared_error(y_test_np, predictions)
                        rmse = np.sqrt(mse)
                        r2 = r2_score(y_test_np, predictions)
                        
                        # Calculate correlation coefficient
                        correlation_matrix = np.corrcoef(y_test_np.flatten(), predictions.flatten())
                        correlation = correlation_matrix[0, 1] if not np.isnan(correlation_matrix[0, 1]) else 0
                        
                        metrics = {
                            'mse': mse,
                            'rmse': rmse,
                            'r2_score': r2,
                            'correlation': correlation
                        }
                
                # Store results
                result = {
                    'run_id': run + 1,
                    'seed': 42 + run,
                    'training_time': training_time,
                    'final_train_loss': train_losses[-1],
                    'final_val_loss': val_losses[-1],
                    'metrics': metrics,
                    'hyperparameters': {
                        'hidden_size': config['hidden_size'],
                        'learning_rate': config['learning_rate'],
                        'weight_decay': config['weight_decay'],
                        'epochs': config['epochs']
                    }
                }
                
                results.append(result)
                
                # Print progress
                if config['task_type'] == 'classification':
                    print(f"  Accuracy: {accuracy:.4f}, F1: {f1:.4f}")
                else:
                    print(f"  R²: {r2:.4f}, RMSE: {rmse:.4f}")
                    
            except Exception as e:
                print(f"Error in run {run + 1}: {str(e)}")
                continue
        
        all_results[config_name] = {
            'algorithm': 'passive_learning',
            'dataset': config['dataset_name'],
            'task_type': config['task_type'],
            'results': results,
            'summary': calculate_summary_stats(results, config['task_type'])
        }
        
        # Save individual results
        filename = f"passive_{config_name.lower().replace(' ', '_')}_multiple_runs.json"
        filepath = f'results/passive_multiple_runs/{filename}'
        
        with open(filepath, 'w') as f:
            json.dump(all_results[config_name], f, indent=2)
        
        print(f"Results saved to {filepath}")
    
    return all_results

def calculate_summary_stats(results, task_type):
    if not results:
        return {}
    
    if task_type == 'classification':
        accuracies = [r['metrics']['accuracy'] for r in results]
        f1_scores = [r['metrics']['f1_score'] for r in results]
        precisions = [r['metrics']['precision'] for r in results]
        recalls = [r['metrics']['recall'] for r in results]
        
        return {
            'accuracy': {
                'mean': np.mean(accuracies),
                'std': np.std(accuracies),
                'min': np.min(accuracies),
                'max': np.max(accuracies)
            },
            'f1_score': {
                'mean': np.mean(f1_scores),
                'std': np.std(f1_scores),
                'min': np.min(f1_scores),
                'max': np.max(f1_scores)
            },
            'precision': {
                'mean': np.mean(precisions),
                'std': np.std(precisions),
                'min': np.min(precisions),
                'max': np.max(precisions)
            },
            'recall': {
                'mean': np.mean(recalls),
                'std': np.std(recalls),
                'min': np.min(recalls),
                'max': np.max(recalls)
            },
            'training_time': {
                'mean': np.mean([r['training_time'] for r in results]),
                'std': np.std([r['training_time'] for r in results])
            }
        }
    else:  # regression
        r2_scores = [r['metrics']['r2_score'] for r in results]
        mse_scores = [r['metrics']['mse'] for r in results]
        rmse_scores = [r['metrics']['rmse'] for r in results]
        correlations = [r['metrics']['correlation'] for r in results]
        
        return {
            'r2_score': {
                'mean': np.mean(r2_scores),
                'std': np.std(r2_scores),
                'min': np.min(r2_scores),
                'max': np.max(r2_scores)
            },
            'mse': {
                'mean': np.mean(mse_scores),
                'std': np.std(mse_scores),
                'min': np.min(mse_scores),
                'max': np.max(mse_scores)
            },
            'rmse': {
                'mean': np.mean(rmse_scores),
                'std': np.std(rmse_scores),
                'min': np.min(rmse_scores),
                'max': np.max(rmse_scores)
            },
            'correlation': {
                'mean': np.mean(correlations),
                'std': np.std(correlations),
                'min': np.min(correlations),
                'max': np.max(correlations)
            },
            'training_time': {
                'mean': np.mean([r['training_time'] for r in results]),
                'std': np.std([r['training_time'] for r in results])
            }
        }

In [ ]:
# Configure all model parameters based on current notebook settings
model_configurations = {
    'Simple_Classification': {
        'dataset_name': 'Breast Cancer (Simple)',
        'task_type': 'classification',
        'input_size': X_train.shape[1],
        'hidden_size': 32,
        'num_classes': sim_num_classes,
        'epochs': 1500,
        'learning_rate': 0.01,
        'weight_decay': 0.0001,
        'data': {
            'X_train': X_train, 'X_val': X_val, 'X_test': X_test,
            'y_train': y_train, 'y_val': y_val, 'y_test': y_test
        }
    },
    'Medium_Classification': {
        'dataset_name': 'Heart Disease (Medium)',
        'task_type': 'classification',
        'input_size': X_train_med.shape[1],
        'hidden_size': 256,
        'num_classes': med_num_classes,
        'epochs': 1000,
        'learning_rate': 0.01,
        'weight_decay': 0.01,
        'data': {
            'X_train': X_train_med, 'X_val': X_val_med, 'X_test': X_test_med,
            'y_train': y_train_med, 'y_val': y_val_med, 'y_test': y_test_med
        }
    },
    'Complex_Classification': {
        'dataset_name': 'Diabetes (Complex)',
        'task_type': 'classification',
        'input_size': X_train_complex.shape[1],
        'hidden_size': 128,
        'num_classes': complex_num_classes,
        'epochs': 1000,
        'learning_rate': 0.005,
        'weight_decay': 0.0001,
        'data': {
            'X_train': X_train_complex, 'X_val': X_val_complex, 'X_test': X_test_complex,
            'y_train': y_train_complex, 'y_val': y_val_complex, 'y_test': y_test_complex
        }
    },
    'Simple_Regression': {
        'dataset_name': 'California Housing (Simple)',
        'task_type': 'regression',
        'input_size': X_train_simple_reg.shape[1],
        'hidden_size': 64,
        'num_classes': None,
        'epochs': 1500,
        'learning_rate': 0.01,
        'weight_decay': 0.01,
        'data': {
            'X_train': X_train_simple_reg, 'X_val': X_val_simple_reg, 'X_test': X_test_simple_reg,
            'y_train': y_train_simple_reg, 'y_val': y_val_simple_reg, 'y_test': y_test_simple_reg
        }
    },
    'Medium_Regression': {
        'dataset_name': 'Real Estate (Medium)',
        'task_type': 'regression',
        'input_size': X_train_med_reg.shape[1],
        'hidden_size': 64,
        'num_classes': None,
        'epochs': 1500,
        'learning_rate': 0.01,
        'weight_decay': 0.01,
        'data': {
            'X_train': X_train_med_reg, 'X_val': X_val_med_reg, 'X_test': X_test_med_reg,
            'y_train': y_train_med_reg, 'y_val': y_val_med_reg, 'y_test': y_test_med_reg
        }
    },
    'Complex_Regression': {
        'dataset_name': 'Comprehensive Housing (Complex)',
        'task_type': 'regression',
        'input_size': X_train_complex_reg.shape[1],
        'hidden_size': 64,
        'num_classes': None,
        'epochs': 1500,
        'learning_rate': 0.05,
        'weight_decay': 0.001,
        'data': {
            'X_train': X_train_complex_reg, 'X_val': X_val_complex_reg, 'X_test': X_test_complex_reg,
            'y_train': y_train_complex_reg, 'y_val': y_val_complex_reg, 'y_test': y_test_complex_reg
        }
    }
}

for name, config in model_configurations.items():
    print(f"  {name}: {config['dataset_name']} - {config['task_type']}")
    print(f"    Hidden: {config['hidden_size']}, LR: {config['learning_rate']}, Epochs: {config['epochs']}")

passive_results = run_multiple_experiments(model_configurations, n_runs=10)

In [ ]:
# Save comprehensive results
comprehensive_results = {
    'experiment_type': 'passive_learning_multiple_runs',
    'timestamp': datetime.now().isoformat(),
    'total_configurations': len(model_configurations),
    'runs_per_configuration': 10,
    'results': passive_results
}

# Save master results file
master_filepath = 'results/passive_multiple_runs/comprehensive_passive_results.json'
with open(master_filepath, 'w') as f:
    json.dump(comprehensive_results, f, indent=2)


for config_name, results in passive_results.items():
    print(f"\n{config_name} ({results['dataset']})")
    print(f"{'-'*50}")
    
    summary = results['summary']
    
    if results['task_type'] == 'classification':
        acc = summary['accuracy']
        f1 = summary['f1_score']
        print(f"  Accuracy:  {acc['mean']:.4f} ± {acc['std']:.4f} (range: {acc['min']:.4f}-{acc['max']:.4f})")
        print(f"  F1 Score:  {f1['mean']:.4f} ± {f1['std']:.4f} (range: {f1['min']:.4f}-{f1['max']:.4f})")
    else:
        r2 = summary['r2_score']
        rmse = summary['rmse']
        print(f"  R² Score:  {r2['mean']:.4f} ± {r2['std']:.4f} (range: {r2['min']:.4f}-{r2['max']:.4f})")
        print(f"  RMSE:      {rmse['mean']:.4f} ± {rmse['std']:.4f} (range: {rmse['min']:.4f}-{rmse['max']:.4f})")
    
    time_stats = summary['training_time']
    print(f"  Avg Time:  {time_stats['mean']:.2f} ± {time_stats['std']:.2f} seconds")
